In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#free test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number + 61)# - 14399)

In [ ]:
#Multiprocessing version.
#mp.cpu_count()
def request_rates(unix_time, base, quote): 
    path='/home/fbuonerba/exchange_rates_data/'
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    while True:
        try:
            exchange=api.exchange_rates_get_specific_rate(base, quote, {'time': utctime})
            with open(path+'exchange_rate_'+str(base)+'_'+str(quote)+'_'+str(unix_time)+'.txt', 'w') as outfile:  
                json.dump(exchange, outfile) 
            break
        except urllib.error.HTTPError as e:
            print(e, unix_time)
            time.sleep(10)
    return(exchange)


In [ ]:
print(time.time())
unix_time= 1530019800
pool = mp.Pool()#processes=7)

for i in range(2):
    print(time.time(), 'start round =', i)
    results = [pool.apply_async(request_rates, args=(unix_time + 600*t,'ETH','USD',)) for t in range(2)]
    output = [res.get() for res in results]
    print(output)
    print(time.time(), 'end round =', i)
    unix_time+=1200

In [ ]:
#class HelloWorld(luigi.Task):
#    def requires(self):
#        return None
#    def output(self):
#        return luigi.LocalTarget('helloworld.txt')
#    def run(self):
#        with self.output().open('w') as outfile:
#            outfile.write('Hello World!\n')

#class NameSubstituter(luigi.Task):
#    name = luigi.Parameter()
#    def requires(self):
#        return HelloWorld()
#    def output(self):
#        return luigi.LocalTarget(self.input().path + '.name_' + self.name)
#    def run(self):
#        with self.input().open() as infile, self.output().open('w') as outfile:
#            text = infile.read()
#            text = text.replace('World', self.name)
#            outfile.write(text)

In [ ]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#Free key for testing: test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number + 61)# - 14399)


In [ ]:
#Luigi version.
#this task downloads data (by luigi default, if it does not exist already).
class request_exchange_rates(luigi.Task):
    unix_time=luigi.Parameter()
    base=luigi.Parameter()
    quote=luigi.Parameter()
    path='/home/fbuonerba/exchange_rates_data/'
    def requires(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.path+'exchange_rate_'+str(self.base)+'_'+str(self.quote)+'_'+str(self.unix_time)+'.txt')
    def run(self):
        while True:
            try: 
                utctime = datetime.utcfromtimestamp(self.unix_time).strftime('%Y-%m-%dT%H:%M:%S')
                exchange=api.exchange_rates_get_specific_rate(self.base, self.quote, {'time': utctime})
                with self.output().open('w') as outfile:
                    json.dump(exchange, outfile) 
                print('okokokokokok', self.unix_time)
                break
            except urllib.error.HTTPError as err:
                print('AAAAAAAAAAAAAAAAAAAA', err, self.unix_time)
                time.sleep(30)
            
        
    

In [ ]:
unix_time= 1529985000
#1586918800  1529954000
while True:
    luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(2)], workers=2, local_scheduler=True)
    unix_time+=1200
    #luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(100)], workers=1, local_scheduler=True)
    

In [ ]:
######GET TOP 50 COINS#######

In [ ]:
usd_sym=[]
for sym in symbols:
    if sym['asset_id_quote']=='USD':
        usd_sym.append(sym['symbol_id'])

usd_sym

In [ ]:
########THIS REQUIRES PLENTY OF REQUESTS#########
#dicti={}
#for sym in usd_sym:
#    print(sym)
#    dicti[sym]=api.ohlcv_latest_data(sym, {'period_id': '1DAY', 'limit':1})

In [ ]:
assets = api.metadata_list_assets()
symbols = api.metadata_list_symbols()

In [ ]:
symbols

In [ ]:
with open('/home/fbuonerba/codes/some_ohlcv.txt') as ff:
    dicti=json.load(ff)

dicti['BITSTAMP_SPOT_BTC_USD']

In [ ]:
btc_sum=0
for sym in symbols:
    if sym['asset_id_quote']=='USD' and sym['asset_id_base']=='BTC' and sym['symbol_type']=='SPOT':
        if dicti[sym['symbol_id']]!=[]:
            btc_sum+= dicti[sym['symbol_id']][0]['volume_traded']
            
                #dicti[sym['symbol_id']][0]['price_high']
btc_sum

In [ ]:
d={}
for asse in assets:
    asse_sum=0
    for sym in symbols:
        if sym['asset_id_quote']=='USD' and sym['asset_id_base']==asse['asset_id'] and sym['symbol_type']=='SPOT':
            s=sym['symbol_id']
            if (s in dicti) and dicti[s]!=[]:
                average=dicti[sym['symbol_id']][0]['price_high']+dicti[sym['symbol_id']][0]['price_low']
                asse_sum=asse_sum + dicti[sym['symbol_id']][0]['volume_traded']*average
    d[asse['asset_id']]=asse_sum


In [ ]:
items = [(v, k) for k, v in d.items()]
items.sort()
items.reverse()             # so largest is first
items = [(k, v) for v, k in items]
top_coins=[fin[0] for fin in items[0:50]]
top_coins

In [ ]:
with open('top_coins.txt','w') as file:
    json.dump(top_coins,file)


In [2]:
exchanges = api.metadata_list_exchanges()


In [5]:
top_exchanges=['BINANCE','BITFINEX','HUOBI','COINBASE','OKEX','MTGOX','GEMINI','BITHUMB','POLONIEX','KRAKEN']

with open('/home/fbuonerba/codes/top_exchanges.txt','w') as file:
    json.dump(top_exchanges,file)